In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt


In [ ]:
# DATA NOTE:
# Parquet file is stored locally and NOT tracked in GitHub
# Update DATA_PATH as needed


df = pd.read_parquet(r"D:\AI Trading - Fin Tech\stock_data_filtered.parquet")
df

In [7]:
df = df.sort_values("MthCalDt").reset_index(drop=True)

import numpy as np

n_chunks = 10
chunks = np.array_split(df, n_chunks)



MemoryError: Unable to allocate 6.64 GiB for an array with shape (234, 3811131) and data type float64

In [ ]:
df